# Pregunta 1

A continuación se definen las utilidades que permiten construir una función de hash:

- **davies_meyer:** Se encarga de generar una función de compresión que recibe mensajes de largo ``l_key + l_message`` y entrega mensajes de largo ``l_message``.
- **pad:** Ajusta los mensajes añadiendo la cantidad de bytes necesaria para que su largo sea múltiplo de ``l_block``. Luego añade un bloque adicional que codifica el largo del mensaje original.
- **merkle_damgard:** Se encarga de generar una función de hash para mensajes de largo arbitrario, haciendo uso de un vector de inicialización ``IV`` y de la función de compresión obtenida anteriormente (aplicada secuencialmente sobre cada bloque del mensaje ajustado por ``pad`` y el resultado respectivo del paso anterior).

In [148]:
# Hashing

# Davies-Meyer construction
def davies_meyer(encrypt, l_key, l_message):
    """
    Arguments:
        encrypt: An encryption function.
        l_key: Length in bytes of the keys for encrypt.
        l_message: Length in bytes of the messages for encrypt.
    Returns:
        A compression function from messages of length (l_key + l_message) to
        messages of length (l_message), defined by using the Davies-Meyer construction.
    """
    # Compression function
    def compress(message):
        enc = encrypt(message[:l_key], message[l_key:])
        return bytearray(enc[i] ^ message[l_key:][i] for i in range(l_message))
    return compress

# Padding
def pad(message, l_block):
    """
    Arguments:
        message: Message to be padded.
        l_block: Length in bytes of the block.
    Returns:
        Extension of message that includes the length of the message
        (in bytes) in its last block.
    """
    padded_message = bytearray(message)
    offset = len(message) % l_block

    # Message needs to be extended
    if offset:
        new_bytes = bytearray.fromhex('01' + (l_block - offset - 1) * '00')
        padded_message += new_bytes

    # Add length block
    padded_message += (len(message) % (2 ** l_block)).to_bytes(l_block, 'big')
    return padded_message

# Merkle-Damgard construction
def merkle_damgard(IV, comp, l_block):
    """
    Arguments:
        IV: Initialization vector for a hash function.
        comp: Compression function to be used in the Merkle-Damgard construction.
        l_block: Length in bytes of the blocks to be used in the Merkle-Damgard construction.
    Returns:
        A hash function for messages of arbitrary length, defined by using
        the Merkle-Damgard construction.
    """
    # Hash function
    def hash_function(message):
        padded = pad(message, l_block)
        h = IV
        for n_block in range(len(padded) // l_block):
            m = padded[l_block * n_block: l_block * (n_block + 1)]
            h = comp(m + h)
        return h
    return hash_function

Se prueba la implementación utilizando el cifrado AES (128-bit) como función de encriptación segura en la construcción de Davies-Meyer, lo que genera la función de compresión utilizada posteriormente en la construcción de Merkle-Damgard:

In [149]:
# Test Hashing
if __name__ == '__main__':
    # AES block cipher (128-bits)
    from Crypto.Cipher import AES
    def AES_128(key, message):
        cipher = AES.new(key, AES.MODE_ECB)
        return bytearray(cipher.encrypt(message))

    # Testing
    compression = davies_meyer(AES_128, 16, 16)
    hash = merkle_damgard(bytearray(b'j12g5g68BnorZOo7'), compression, 16)
    s0 = bytearray(b'')
    s1 = bytearray(b'Roger was known as Pirate King..')
    s2 = bytearray(b'Roger was known')
    s3 = bytearray(b'Roger was known as Pirate King.')
    s4 = bytearray(b'Gol D. Roger was known as the Pirate King, the strongest and most infamous being to have sailed the Grand Line. The capture and execution of Roger by the World Government brought a change throughout the world. His last words before his death revealed the existence of the greatest treasure in the world, One Piece. It was this revelation that brought about the Grand Age of Pirates, men who dreamed of finding One Piece-which promises an unlimited amount of riches and fame-and quite possibly the pinnacle of glory and the title of the Pirate King.')
    h0 = hash(s0)
    h1 = hash(s1)
    h2 = hash(s2)
    h3 = hash(s3)
    h4 = hash(s4)
    print(h0.hex().upper())
    print(h1.hex().upper())
    print(h2.hex().upper())
    print(h3.hex().upper())
    print(h4.hex().upper())

CFADB5C4677D641FACBE14316747E831
778C6C3E3C4FF47A5B529CE2C2506758
508484D8D946F907C8F6F0FD4722FECD
22629910D3B060A67EA8748A4D85F0C3
A1C96608BC19FACC466B2428894FF326
